In [27]:
import boto3
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pickle

In [13]:
pkl_dir = "pickle_files"
df = pd.read_csv('dataset/creditcard.csv')
df_fe = df.drop(['V8','V13','V15','V20','V22','V23','V24','V25','V26','V27','V28'], axis =1)

In [15]:
def undersample(data, multiple):  # multiple denote the normal data = multiple * fraud data
    count_fraud_transaction = len(data[data["Class"] == 1])  # fraud by 1
    fraud_indices = np.array(data[data.Class == 1].index)
    normal_indices = np.array(data[data.Class == 0].index)
    normal_indices_undersample = np.array(
        np.random.choice(normal_indices, (multiple * count_fraud_transaction), replace=False))
    undersample_data = np.concatenate([fraud_indices, normal_indices_undersample])
    undersample_data = data.iloc[undersample_data, :]
    return undersample_data

In [20]:
undersample_data_svm = undersample(df_fe, 8)
X_undersample_svm = undersample_data_svm.iloc[:, undersample_data_svm.columns != "Class"]
y_undersample_svm = undersample_data_svm.iloc[:, undersample_data_svm.columns == "Class"]
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X_undersample_svm, y_undersample_svm, random_state=0)

In [33]:
svm_columns = df_fe.columns.tolist()[:-1]
svm_columns_pklfile = pkl_dir + '/svm_columns.pkl'

svm_model = SVC(C=1, gamma=0.1)
svm_model.fit(X_train_svm, y_train_svm.values.ravel())
svm_model_pklfile = pkl_dir + '/svm_model.pkl'

with open(svm_columns_pkfile, "wb") as fp:  
    pickle.dump(svm_columns, fp)
print "svm_columns pickled!"

with open(svm_model_pklfile, "wb") as fp:  
    pickle.dump(svm_model, fp)
print "svm_model pickled!"

svm_columns pickled!
svm_model pickled!


In [34]:
undersample_data_rf = undersample(df_fe, 70)
X_undersample_rf = undersample_data_rf.iloc[:, undersample_data_rf.columns != "Class"]
y_undersample_rf = undersample_data_rf.iloc[:, undersample_data_rf.columns == "Class"]
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_undersample_rf, y_undersample_rf, random_state=0)

In [35]:
rf_columns = df_fe.columns.tolist()[:-1]
rf_columns_pklfile = pkl_dir + '/random_forest_columns.pkl'

rf_model = RandomForestClassifier(n_estimators=200, max_features=1, max_depth=20, min_samples_split=2, min_samples_leaf=1)
rf_model.fit(X_train_rf, y_train_rf.values.ravel())
rf_model_pklfile = pkl_dir + '/random_forest_model.pkl'

with open(rf_columns_pkfile, "wb") as fp:  
    pickle.dump(rf_columns, fp)
print "rf_columns pickled!"

with open(rf_model_pklfile, "wb") as fp:  
    pickle.dump(rf_model, fp)
print "rf_model pickled!"

rf_columns pickled!
rf_model pickled!


In [38]:
try:
    S3 = boto3.client('s3', region_name='us-east-1')
    print "Connected to S3!"
except:
    print "Fail to connect to S3"

bucket_name = 'info7390-2018spring-team2-final-models'

try:
    S3.upload_file(rf_columns_pklfile, bucket_name, rf_columns_pklfile.split('/')[1])
    print "rf_columns uploaded!"
    S3.upload_file(rf_model_pklfile, bucket_name, rf_model_pklfile.split('/')[1])
    print "rf_model uploaded!"
    S3.upload_file(svm_columns_pklfile, bucket_name, svm_columns_pklfile.split('/')[1])
    print "svm_columns uploaded!"
    S3.upload_file(svm_model_pklfile, bucket_name, svm_model_pklfile.split('/')[1])
    print "svm_model uploaded!"
except:
    print "Fail to upload file!"

Connected to S3!
rf_columns uploaded!
rf_model uploaded!
svm_columns uploaded!
svm_model uploaded!
